## Классификация текстов с использованием предобученных языковых моделей.

В данном задании вам предстоит обратиться к задаче классификации текстов и решить ее с использованием предобученной модели BERT.

In [ ]:
!pip install transformers torch

In [ ]:
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

import json
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from IPython import display
from nltk.tokenize import WordPunctTokenizer
from sklearn import naive_bayes
from sklearn.base import BaseEstimator
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from torch import nn
from torch.nn import functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR

import json
from nltk.tokenize import WordPunctTokenizer
from collections import Counter
from sklearn.linear_model import LogisticRegression
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import clear_output
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

%matplotlib inline

Обратимся к набору данных SST-2. Holdout часть данных (которая понадобится вам для посылки) доступна по ссылке ниже.

In [ ]:


!wget https://raw.githubusercontent.com/girafe-ai/ml-course/refs/heads/24f_yandex_ml_trainings/homeworks/hw04_bert_and_co/texts_holdout.json


--2025-04-21 18:26:18--  https://raw.githubusercontent.com/girafe-ai/ml-course/refs/heads/24f_yandex_ml_trainings/homeworks/hw04_bert_and_co/texts_holdout.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51581 (50K) [text/plain]
Saving to: ‘texts_holdout.json.2’

texts_holdout.json. 100%[===================>]  50.37K  --.-KB/s    in 0.007s  

2025-04-21 18:26:18 (6.70 MB/s) - ‘texts_holdout.json.2’ saved [51581/51581]



In [ ]:

df = pd.read_csv(
    "https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv",
    delimiter="\t",
    header=None,
)
texts_train = df[0].values[:5000]
y_train = df[1].values[:5000]
texts_test = df[0].values[5000:]
y_test = df[1].values[5000:]
with open("texts_holdout.json") as iofile:
    texts_holdout = json.load(iofile)


Весь остальной код предстоит написать вам.

Для успешной сдачи на максимальный балл необходимо добиться хотя бы __84.5% accuracy на тестовой части выборки__.

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
# 1. Загрузка предобученной модели BERT и токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

In [ ]:
# Создание датасета
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=256):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0)
        }


In [ ]:
# 3. Функция для получения эмбеддингов из BERT
def get_bert_embeddings(texts, model, tokenizer, batch_size=64, max_length=256):
    dataset = TextDataset(texts, tokenizer, max_length)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Generating embeddings"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            # Используем [CLS] токен как представление всего текста
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(cls_embeddings)

    return np.vstack(embeddings)


In [ ]:
# 4. Получение эмбеддингов для train, test и holdout
X_train_emb = get_bert_embeddings(texts_train, bert_model, tokenizer)
X_test_emb = get_bert_embeddings(texts_test, bert_model, tokenizer)
X_holdout_emb = get_bert_embeddings(texts_holdout, bert_model, tokenizer)

# Преобразование в тензоры PyTorch
X_train_emb_torch = torch.tensor(X_train_emb, dtype=torch.float32).to(device)
X_test_emb_torch = torch.tensor(X_test_emb, dtype=torch.float32).to(device)
X_holdout_emb_torch = torch.tensor(X_holdout_emb, dtype=torch.float32).to(device)

y_train_torch = torch.tensor(y_train, dtype=torch.long).to(device)
y_test_torch = torch.tensor(y_test, dtype=torch.long).to(device)



Generating embeddings: 100%|██████████| 8/8 [00:07<00:00,  1.05it/s]


In [ ]:
# 5. Определение классификатора
class Classifier(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Размерность входных данных (размер эмбеддинга BERT)
input_dim = X_train_emb.shape[1]
hidden_dim = 256

model = Classifier(input_dim, hidden_dim).to(device)


In [ ]:
# 6. Оптимизатор и функция потерь
loss_function = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# 7. Функция обучения модели
def train_model(model, opt, X_train, y_train, X_test, y_test, n_iterations=150):
    for i in tqdm(range(n_iterations), desc="Training"):
        model.train()
        opt.zero_grad()

        # Forward pass
        outputs = model(X_train)
        loss = loss_function(outputs, y_train)

        # Backward pass
        loss.backward()
        opt.step()

        # Логирование каждые 100 итераций
        if (i + 1) % 100 == 0:
            model.eval()
            with torch.no_grad():
                train_preds = torch.argmax(outputs, dim=1)
                train_acc = accuracy_score(y_train.cpu(), train_preds.cpu())

                test_outputs = model(X_test)
                test_preds = torch.argmax(test_outputs, dim=1)
                test_acc = accuracy_score(y_test.cpu(), test_preds.cpu())

            print(f"Iteration {i+1}: Train Acc = {train_acc:.4f}, Test Acc = {test_acc:.4f}")

    return model


In [ ]:
# 8. Обучение модели
model = train_model(model, opt, X_train_emb_torch, y_train_torch, X_test_emb_torch, y_test_torch)

Training: 100%|██████████| 150/150 [00:00<00:00, 559.92it/s]

Iteration 100: Train Acc = 0.9162, Test Acc = 0.8526


In [ ]:
# 9. Предсказания вероятностей
def predict_proba(model, X):
    model.eval()
    with torch.no_grad():
        outputs = model(X)
        probs = F.softmax(outputs, dim=1)[:, 1].cpu().numpy()  # Вероятности класса 1
    return probs

train_probs = predict_proba(model, X_train_emb_torch)
test_probs = predict_proba(model, X_test_emb_torch)
holdout_probs = predict_proba(model, X_holdout_emb_torch)


#### Сдача взадания в контест


In [ ]:
out_dict = {
    'train': train_probs, # list of length 5000 with probas
    'test': test_probs, # list of length 1920 with probas
    'holdout': holdout_probs # list of length 500 with probas
}

In [ ]:
# Преобразование в список, если это массив NumPy или тензор PyTorch
if isinstance(out_dict["train"], np.ndarray):
    out_dict["train"] = out_dict["train"].tolist()
elif isinstance(out_dict["train"], torch.Tensor):
    out_dict["train"] = out_dict["train"].cpu().numpy().tolist()

# Преобразование всех элементов в float
out_dict["train"] = [float(x) for x in out_dict["train"]]

# Повторите аналогичные шаги для test и holdout
if isinstance(out_dict["test"], np.ndarray):
    out_dict["test"] = out_dict["test"].tolist()
elif isinstance(out_dict["test"], torch.Tensor):
    out_dict["test"] = out_dict["test"].cpu().numpy().tolist()
out_dict["test"] = [float(x) for x in out_dict["test"]]

if isinstance(out_dict["holdout"], np.ndarray):
    out_dict["holdout"] = out_dict["holdout"].tolist()
elif isinstance(out_dict["holdout"], torch.Tensor):
    out_dict["holdout"] = out_dict["holdout"].cpu().numpy().tolist()
out_dict["holdout"] = [float(x) for x in out_dict["holdout"]]

Несколько `assert`'ов для проверки вашей посылки:

In [ ]:
assert isinstance(out_dict["train"], list), "Object must be a list of floats"
assert isinstance(out_dict["train"][0], float), "Object must be a list of floats"
assert (
    len(out_dict["train"]) == 5000
), "The predicted probas list length does not match the train set size"

assert isinstance(out_dict["test"], list), "Object must be a list of floats"
assert isinstance(out_dict["test"][0], float), "Object must be a list of floats"
assert (
    len(out_dict["test"]) == 1920
), "The predicted probas list length does not match the test set size"

assert isinstance(out_dict["holdout"], list), "Object must be a list of floats"
assert isinstance(out_dict["holdout"][0], float), "Object must be a list of floats"
assert (
    len(out_dict["holdout"]) == 500
), "The predicted probas list length does not match the holdout set size"

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate Test Accuracy
threshold = 0.5
y_pred = [1 if p >= threshold else 0 for p in test_probs]
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.8458


На этом задание завершено. Поздравляем!